In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Alzheimers_Disease"
cohort = "GSE214417"

# Input paths
in_trait_dir = "../../input/GEO/Alzheimers_Disease"
in_cohort_dir = "../../input/GEO/Alzheimers_Disease/GSE214417"

# Output paths
out_data_file = "../../output/preprocess/Alzheimers_Disease/GSE214417.csv"
out_gene_data_file = "../../output/preprocess/Alzheimers_Disease/gene_data/GSE214417.csv"
out_clinical_data_file = "../../output/preprocess/Alzheimers_Disease/clinical_data/GSE214417.csv"
json_path = "../../output/preprocess/Alzheimers_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Long-term Urolithin A treatment ameliorates disease pathology in Alzheimer's Disease mouse models"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['tissue: hippocampus'], 1: ['treatment: water', 'treatment: Urolithin A_5m', 'treatment: water+washout', 'treatment: Urolithin A_5m+washout_1m'], 2: ['Sex: Male'], 3: ['age: 8 months', 'age: 9 months'], 4: ['strain: B6.Cg-Tg(APPswe,PSEN1dE9)85Dbo/J'], 5: ['genotype: - APP - PSEN', 'genotype: + APP + PSEN']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Looking at the background information, this appears to be gene expression data from mouse models of Alzheimer's disease
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait, we can use the genotype information (key 5) which distinguishes AD vs control mice
trait_row = 5  # 'genotype: - APP - PSEN' vs 'genotype: + APP + PSEN'

# Age information is available at key 3
age_row = 3  # 'age: 8 months', 'age: 9 months'

# Gender information is available at key 2, but it's a constant (all Male)
gender_row = None  # Only one value 'Sex: Male', so it's not useful for association studies

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    if value is None:
        return None
    # Extract the part after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # The genotype with "+" indicates Alzheimer's disease model, "-" indicates control
    if '+ APP + PSEN' in value:
        return 1  # AD model
    elif '- APP - PSEN' in value:
        return 0  # Control
    else:
        return None

def convert_age(value):
    if value is None:
        return None
    # Extract the part after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Extract the numeric age in months
    if 'months' in value or 'month' in value:
        try:
            # Extract just the number
            age_value = value.split()[0]
            return float(age_value)
        except (ValueError, IndexError):
            return None
    return None

# Not used but defined for completeness
def convert_gender(value):
    if value is None:
        return None
    # Extract the part after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    else:
        return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Assume clinical_data is already defined from a previous step
    # If not, we would need to load it
    try:
        # Load clinical data if it's not already available
        clinical_data_file = os.path.join(in_cohort_dir, "clinical_data.csv")
        if 'clinical_data' not in locals():
            clinical_data = pd.read_csv(clinical_data_file)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age
            # gender_row and convert_gender are None since gender is constant
        )
        
        # Preview the extracted clinical features
        preview = preview_df(clinical_features)
        print("Clinical Features Preview:")
        print(preview)
        
        # Create the directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical features to a CSV file
        clinical_features.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical features saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error in clinical feature extraction: {str(e)}")


Clinical Features Preview:
{'GSM6567822': [0.0, 8.0], 'GSM6567823': [0.0, 8.0], 'GSM6567824': [0.0, 8.0], 'GSM6567825': [0.0, 8.0], 'GSM6567826': [1.0, 8.0], 'GSM6567827': [1.0, 8.0], 'GSM6567828': [1.0, 8.0], 'GSM6567829': [1.0, 8.0], 'GSM6567830': [1.0, 8.0], 'GSM6567831': [1.0, 8.0], 'GSM6567832': [1.0, 8.0], 'GSM6567833': [0.0, 9.0], 'GSM6567834': [0.0, 9.0], 'GSM6567835': [0.0, 9.0], 'GSM6567836': [0.0, 9.0], 'GSM6567837': [0.0, 9.0], 'GSM6567838': [1.0, 9.0], 'GSM6567839': [1.0, 9.0], 'GSM6567840': [1.0, 9.0], 'GSM6567841': [1.0, 9.0], 'GSM6567842': [1.0, 9.0], 'GSM6567843': [1.0, 9.0], 'GSM6567844': [1.0, 9.0], 'GSM6567845': [1.0, 9.0]}
Clinical features saved to ../../output/preprocess/Alzheimers_Disease/clinical_data/GSE214417.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers are numeric values (1, 2, 3, etc.) and not human gene symbols
# These appear to be row numbers or possibly probe IDs that need to be mapped to actual gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': ['328', '326', '324', '322', '320'], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_51_P399985', 'A_55_P2508138'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'CONTROL', nan, nan], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_015742', 'NR_028378'], 'GB_ACC': [nan, nan, nan, 'NM_015742', 'NR_028378'], 'LOCUSLINK_ID': [nan, nan, nan, 17925.0, 100034739.0], 'GENE_SYMBOL': [nan, nan, nan, 'Myo9b', 'Gm17762'], 'GENE_NAME': [nan, nan, nan, 'myosin IXb', 'predicted gene, 17762'], 'UNIGENE_ID': [nan, nan, nan, 'Mm.33779', 'Mm.401643'], 'ENSEMBL_ID': [nan, nan, nan, 'ENSMUST00000170242', nan], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_015742|ref|NM_001142322|ref|NM_001142323|ens|ENSMUST00000170242', 'ref|NR_028378|gb|AK171729|gb|AK045818|gb|AK033161'], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, 'chr8:73884459-73884518', 'chr2:17952143-17952202'

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the appropriate columns in gene_annotation for mapping
# The gene expression data uses numeric identifiers that correspond to the 'ID' column
# The gene symbols are stored in the 'GENE_SYMBOL' column

# 2. Create gene mapping dataframe using the get_gene_mapping function
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'GENE_SYMBOL')

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# The apply_gene_mapping function handles the many-to-many relationship as described
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the mapped gene expression data
print("Mapped gene expression data (first 5 genes):")
print(gene_data.head(5))
print(f"Shape of mapped gene expression data: {gene_data.shape}")


Mapped gene expression data (first 5 genes):
            GSM6567822  GSM6567823  GSM6567824  GSM6567825  GSM6567826  \
Gene                                                                     
A130033P14       -0.24       -0.18       -0.21       -0.30       -0.23   
A230055C15        0.25        0.45        0.41        0.32        0.39   
A330044H09        0.85        0.91        0.90        0.78        0.90   
A430057O09       -1.21       -1.04       -1.26       -1.15       -1.19   
A430085C19       -0.68       -0.95       -0.83       -0.89       -0.92   

            GSM6567827  GSM6567828  GSM6567829  GSM6567830  GSM6567831  ...  \
Gene                                                                    ...   
A130033P14       -0.18       -0.20       -0.19       -0.16       -0.18  ...   
A230055C15        0.33        0.37        0.39        0.41        0.41  ...   
A330044H09        0.91        0.87        0.90        0.82        0.91  ...   
A430057O09       -1.22       -1.09       

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from bronchial brushings from control individuals and patients with asthma after rhinovirus infection in vivo, as described in the study 'Rhinovirus-induced epithelial RIG-I inflammasome suppresses antiviral immunity and promotes inflammation in asthma and COVID-19'."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...


Gene data shape after normalization: (30, 24)
Normalized gene data saved to ../../output/preprocess/Alzheimers_Disease/gene_data/GSE214417.csv
Loading the original clinical data...
Extracting clinical features...
Clinical data preview:
{'GSM6567822': [0.0, 8.0], 'GSM6567823': [0.0, 8.0], 'GSM6567824': [0.0, 8.0], 'GSM6567825': [0.0, 8.0], 'GSM6567826': [1.0, 8.0], 'GSM6567827': [1.0, 8.0], 'GSM6567828': [1.0, 8.0], 'GSM6567829': [1.0, 8.0], 'GSM6567830': [1.0, 8.0], 'GSM6567831': [1.0, 8.0], 'GSM6567832': [1.0, 8.0], 'GSM6567833': [0.0, 9.0], 'GSM6567834': [0.0, 9.0], 'GSM6567835': [0.0, 9.0], 'GSM6567836': [0.0, 9.0], 'GSM6567837': [0.0, 9.0], 'GSM6567838': [1.0, 9.0], 'GSM6567839': [1.0, 9.0], 'GSM6567840': [1.0, 9.0], 'GSM6567841': [1.0, 9.0], 'GSM6567842': [1.0, 9.0], 'GSM6567843': [1.0, 9.0], 'GSM6567844': [1.0, 9.0], 'GSM6567845': [1.0, 9.0]}
Clinical data saved to ../../output/preprocess/Alzheimers_Disease/clinical_data/GSE214417.csv
Linking clinical and genetic data...
Linked d